In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from scipy.stats import linregress

Phi     =         # Ph inicial
M_sus   =         #g
C_Naoh  =         # normalidad de la base (determinada por contraste como se indica a continuación)
V_Naoh  =         #mL
mp      =         # masa de proteína, g (masa pesada • 0,77)
alpha   =         # grado de disociación (α = 0,885 para pH = 8 y T = 30-50ºC
h_total =         # número de enlaces peptídicos en la proteína (8,8 eq-g/kg)
V_bini  =         # volumen bureta inicial mL
V_mez   =         # volumen de la mezcla, L
C_0     = M_sus / V_mez
# Datos tomados

Datos = {
    'tiempo, s' : np.array([]),                       # Lista vacía para el tiempo en segundos
    'volumen Naoh gastado, mL' : np.array([]),        # Lista vacía para volumen de NaOH gastado
    'pH' : np.array([])                               # Lista vacía para pH medido
}
Datos['tiempo, min'] = Datos['tiempo, s'] / 60
Datos['volumen de base gastado ,L'] = Datos['volumen Naoh gastado, mL'] / 1000
Datos['volumen de bureta, mL'] = V_bini - Datos['volumen Naoh gastado, mL']
Datos['DH'] = 100 * (Datos['volumen de base gastado ,L'] * C_Naoh) / (alpha * mp * h_total)

pd_datos = pd.DataFrame(Datos)
display(pd_datos)
# Gráfica
plt.plot(Datos['tiempo, min'], Datos['DH'])
plt.xlabel('tiempo, min')
plt.ylabel('Grado de hidrolisis, %')
plt.title('Grado de hidrolisis vs tiempo')
          
# calculo de pendientes de la gráfica
interceptos = []
r_values = []
pendientes = []
std_errs = []


for i in range(len(Datos['tiempo, min']) - 1):
    # Calcular la pendiente entre el punto i y el punto i+1
    slope, intercept, r_value, _, std_err = linregress(
        Datos['tiempo, min'][i:i+2],  # Dos primeros puntos de tiempo
        Datos['DH'][i:i+2]            # Dos primeros puntos de DH
    )
    pendientes.append(slope)  # Almacenar la pendiente
    interceptos.append(intercept)
    r_values.append(r_value)
    std_errs.append(std_err)

# Crear un DataFrame con las pendientes calculadas
pd_calculo = pd.DataFrame({
    'pendiente': pendientes,
    'intercepto': interceptos,
    'r_value': r_values,
    'std_err': std_errs
})
display(pd_calculo)
# Calcular masa de proteína hidrolizada
M_h = M_sus * (DH / 100)  # Masa de proteína hidrolizada en g

# Calcular concentración de sustrato remanente
concentraciones_sustrato  = C_0 - (M_h / V_reactor)  # Concentración remanente en g/L
velocidades_iniciales = pd_calculo['pendiente']  
# Definir la ecuación de Michaelis-Menten
def michaelis_menten(S, Vmax, Km):
    return (Vmax * S) / (Km + S)

# Datos experimentales
concentraciones_sustrato = [...]  # Lista de concentraciones de sustrato [S]
velocidades_iniciales = pd_calculo['pendiente']  

# Ajustar los datos a la ecuación de Michaelis-Menten
popt, pcov = curve_fit(michaelis_menten, concentraciones_sustrato, velocidades_iniciales)

# Los parámetros ajustados son Vmax y Km
Vmax, Km = popt
# Calcular los recíprocos de [S] y V_0
inv_S = 1 / concentraciones_sustrato
inv_V0 = 1 / velocidades_iniciales

# Ajuste de la línea recta (Lineweaver-Burk)
slope, intercept, r_value, p_value, std_err = linregress(inv_S, inv_V0)

# Obtener Vmax y Km a partir de la pendiente e intercepto
Vmax2 = 1 / intercept
Km2 = slope * Vmax
# Crear subplots para Michaelis-Menten y Lineweaver-Burk
fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # 1 fila, 2 columnas

# Gráfica de Michaelis-Menten (subplot 1)
S_fit = np.linspace(0, max(concentraciones_sustrato), 100)
V_fit = michaelis_menten(S_fit, Vmax, Km)

axs[0].scatter(concentraciones_sustrato, velocidades_iniciales, color='blue', label='Datos experimentales')
axs[0].plot(S_fit, V_fit, color='red', label='Ajuste Michaelis-Menten')
axs[0].set_xlabel('Concentración de sustrato [S] (g/L)')
axs[0].set_ylabel('Velocidad de la reacción V_0 (g/L*s)')
axs[0].set_title('Curva de Michaelis-Menten')
axs[0].legend()
axs[0].grid(True)

# Gráfica de Lineweaver-Burk (subplot 2)
axs[1].scatter(inv_S, inv_V0, label='Datos experimentales', color='blue')
axs[1].plot(inv_S, slope * inv_S + intercept, color='red', label='Ajuste Lineweaver-Burk')
axs[1].set_xlabel('1/[S] (L/g)')
axs[1].set_ylabel('1/V_0 (s*g/L)')
axs[1].set_title('Gráfica de Lineweaver-Burk')
axs[1].legend()
axs[1].grid(True)

# Mostrar las dos gráficas
plt.tight_layout()
plt.show()


#tabla vmax y km

tabla_vmax_km = pd.DataFrame({
    'ajuste': ['Michaelis-Menten', 'Lineweaver-Burk'],
    'Vmax (g/L*s)': [Vmax, Vmax2],
    'Km (g/L)': [Km, Km2]
})
display(tabla_vmax_km)
